**Question:** What is the differences between NumbaPro and Numba?

[Answer](https://docs.anaconda.com/numbapro/):  
NumbaPro has been deprecated, and its code generation features have been moved into open-source Numba. The CUDA library functions have been moved into Accelerate, along with some Intel MKL functionality. High-level functions and access to additional native library implementations will be added in future releases of Accelerate, and there will be no further updates to NumbaPro.

**Question:** However, I could not found the package called Accelerate

[Answer](https://docs.anaconda.com/accelerate/)
This product is **discontinued**. This archived copy of the product documentation is provided for those customers who are still using it. SEE https://www.anaconda.com/blog/developer-blog/open-sourcing-anaconda-accelerate/

On July 27, 2017, **Accelerate was split into the Intel Distribution for Python and the open source Numba project’s sub-projects** pyculib, pyculib_sorting and data_profiler. Numba and all its sub-projects are available under a BSD license.

**Question:** Is there still any reason to learn julia even when we have numba, jit, ... for python?

[Answer](https://discourse.julialang.org/t/julia-motivation-why-werent-numpy-scipy-numba-good-enough/2236/7)  
There are a lot of discussion at this forum. However, I believe Julia will one day expand. Therefore, it is better to learn it.

# Timeit magic

In [4]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%python

line magic to time a line of code

In [5]:
%timeit 3 + 1

8.13 ns ± 0.0169 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [6]:
def add2nums(a, b):
    return a + b

In [7]:
%timeit add2nums(3, 1)

61.4 ns ± 0.511 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


cell magic version of time it

In [8]:
%%timeit
a = [4 + i for i in range(1000)]

32.8 µs ± 363 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
%%timeit
a = []
for i in range(1000):
    a.append(4 + i)

59 µs ± 533 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
import numpy as np

In [11]:
%%timeit
4 + np.arange(1000)

2.03 µs ± 18 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# First try on Numba --- Finding Prime Number

## Write a function isPrime to check if a number is prime or not

In [37]:
import math

In [26]:
def isPrime(num):
    """ Checking whether the input integer is a prime number or not
    
    Args:
        num (int): integer specified from user
        
    Return:
        bool: True if the num is a prime; False otherwise
    """
    if (num == 2) or (num == 3) or (num == 5):
        return True
    
    if num % 2 == 0:
        return False
    if num % 3 == 0:
        return False
    if num % 5 == 0:
        return False
    
    # max divisor
    max_div = int( math.ceil(math.sqrt(num)) )
    num_div = 5
    
    while num_div <= max_div:
        if num % num_div == 0:
            return False
        num_div += 2
        
    return True

check if the function works correctly 

In [27]:
# Special Cases
print('{:2} {}'.format(2, isPrime(2)))
print('{:2} {}'.format(3, isPrime(3)))
print('{:2} {}'.format(5, isPrime(5)))
print("==============================")
# Prime Numbers
print('{:2} {}'.format(11, isPrime(11)))
print('{:2} {}'.format(13, isPrime(13)))
print('{:2} {}'.format(17, isPrime(17)))
print('{:2} {}'.format(23, isPrime(23)))
print("==============================")
# Not Prime Numbers
print('{:2} {}'.format( 4, isPrime(4)))
print('{:2} {}'.format( 6, isPrime(6)))
print('{:2} {}'.format(10, isPrime(10)))
print('{:2} {}'.format(21, isPrime(21)))

 2 True
 3 True
 5 True
11 True
13 True
17 True
23 True
 4 False
 6 False
10 False
21 False


## Use decorator jit to compile the function using LLVM

In [33]:
from numba import jit

In [34]:
@jit
def isPrime_jit(num):
    """ Checking whether the input integer is a prime number or not
    
    Args:
        num (int): integer specified from user
        
    Return:
        bool: True if the num is a prime; False otherwise
    """
    if (num == 2) or (num == 3) or (num == 5):
        return True
    
    if num % 2 == 0:
        return False
    if num % 3 == 0:
        return False
    if num % 5 == 0:
        return False
    
    # max divisor
    max_div = int( math.ceil(math.sqrt(num)) )
    num_div = 5
    
    while num_div <= max_div:
        if num % num_div == 0:
            return False
        num_div += 2
        
    return True

## Time comparison

In [35]:
%%timeit 
x = [isPrime(num) for num in range(1, 100)]

29 µs ± 256 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [36]:
%%timeit 
x = [isPrime_jit(num) for num in range(1, 100)]

15.7 µs ± 3.99 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


# GPU

In [3]:
from numba import cuda

In [4]:
from numba import hsa

ValueError: NUMBA_HSA_DRIVER /opt/hsa/lib/libhsa-runtime64.so is not a valid file path.  Note it must be a filepath of the .so/.dll/.dylib or the driver